In [9]:
import torch
import random
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.dynamic_module_utils import get_imports
from datasets import load_dataset
from unittest.mock import patch
import os


def fixed_get_imports(filename: str | os.PathLike) -> list[str]:
    imports = get_imports(filename)
    if not torch.cuda.is_available() and "flash_attn" in imports:
        imports.remove("flash_attn")
    return imports

class DeepSeekMMLUEvaluation:
    def __init__(self, model_name="deepseek-ai/DeepSeek-R1", dataset_name="hendrycks_test", subset="mmlu"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model_name = 'qnguyen3/nanoLLaVA-1.5'
        with patch("transformers.dynamic_module_utils.get_imports", fixed_get_imports):
            self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    device_map='auto' if torch.cuda.is_available() else None,
                    trust_remote_code=True
                )
            self.tokenizer = AutoTokenizer.from_pretrained(
                        model_name,
                        trust_remote_code=True
                    )
        self.dataset = load_dataset(dataset_name, subset)
    
    def preprocess(self, text):
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
        return inputs
    
    def evaluate(self, split="validation"):
        dataset_split = self.dataset[split]
        correct = 0
        total = len(dataset_split)
        
        for item in dataset_split:
            question = item["question"]
            choices = item["choices"]
            correct_answer = item["answer"]
            
            inputs = self.preprocess(question + "\n" + "\n".join(choices))
            with torch.no_grad():
                outputs = self.model(**inputs)
            logits = outputs.logits[:, -1, :]
            predicted_answer = torch.argmax(logits).item()
            
            if predicted_answer == correct_answer:
                correct += 1
        
        accuracy = correct / total * 100
        return accuracy
    
    def inject_random_bit_flip(self):
        params = list(self.model.named_parameters())
        param_name, param_tensor = random.choice(params)
        param_data = param_tensor.detach().cpu().numpy()
        param_flat = param_data.flatten()
        
        byte_index = random.randint(0, param_flat.size - 1)
        bit_position = random.randint(0, 7)
        
        param_flat[byte_index] ^= (1 << bit_position)
        modified_param = torch.tensor(param_flat.reshape(param_data.shape), dtype=param_tensor.dtype).to(self.device)
        
        param_tensor.data = modified_param
        return param_name, byte_index, bit_position
    
    def run_experiment(self, num_flips=5):
        original_accuracy = self.evaluate()
        print(f"Original MMLU Accuracy: {original_accuracy:.2f}%")
        
        flip_results = []
        for i in range(num_flips):
            param_name, byte_index, bit_position = self.inject_random_bit_flip()
            new_accuracy = self.evaluate()
            flip_results.append((param_name, byte_index, bit_position, new_accuracy))
            print(f"Flip {i+1}: {param_name}, Byte {byte_index}, Bit {bit_position}, Accuracy: {new_accuracy:.2f}%")
        
        return flip_results


In [10]:
deepseek=DeepSeekMMLUEvaluation()
deepseek.run_experiment()

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

configuration_llava_qwen2.py:   0%|          | 0.00/8.87k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/qnguyen3/nanoLLaVA-1.5:
- configuration_llava_qwen2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_llava_qwen2.py:   0%|          | 0.00/103k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/qnguyen3/nanoLLaVA-1.5:
- modeling_llava_qwen2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


KeyError: 'tags'